# Module 1: Data Preprocessing
The weather folder consists of the weather data at 15
different airports in the USA between 2016 and 2017. The flight data of all the flights that flew
inside the USA is provided. Merge these two data appropriately such that each record of the
flight should have the corresponding weather data available.


## 1. Imports

In [ ]:
!python -m pip install 'fsspec>=0.3.3'

import pandas as pd
import numpy as np
from dask import delayed
import dask.dataframe as dd
import os
import json
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Utilities

In [ ]:
basePath = "drive/MyDrive/Data"

airports = ["ATL", "CLT", "DEN", "DFW", "EWR", "IAH", "JFK", "LAS", "LAX", "MCO", "MIA", "ORD", "PHX", "SEA", "SFO"]

flightCols = ["Origin", "Dest", "FlightDate", "Quarter", "Year", "Month", "DayofMonth", "DepTime", "DepDel15", "CRSDepTime", "DepDelayMinutes", "OriginAirportID", "DestAirportID", "ArrTime", "CRSArrTime", "ArrDel15", "ArrDelayMinutes"]

weatherCols = ["Date", "Month", "Year", "Airport", "WindSpeedKmph", "WindDirDegree", "WeatherCode", "precipMM", "Visibility", "Pressure", "Cloudcover", "DewPointF", "WindGustKmph", "tempF", "WindChillF", "Humidity", "Time"]

weatherKeys = ["windspeedKmph", "winddirDegree", "weatherCode", "precipMM", "visibility", "pressure", "cloudcover", "DewPointF", "WindGustKmph", "tempF", "WindChillF", "humidity", "time"]

flightColsW = ["OriginWindSpeedKmph", "OriginWindDirDegree", "OriginWeatherCode", "OriginprecipMM", "OriginVisibility", "OriginPressure", "OriginCloudcover", "OriginDewPointF", "OriginWindGustKmph", "OrigintempF", "OriginWindChillF", "OriginHumidity", "OriginTime", "DestWindSpeedKmph", "DestWindDirDegree", "DestWeatherCode", "DestprecipMM", "DestVisibility", "DestPressure", "DestCloudcover", "DestDewPointF", "DestWindGustKmph", "DesttempF", "DestWindChillF", "DestHumidity", "DestTime"]

years = ['2016', '2017']

months = range(1, 13)

merge_right = ["Year", "Month", "Date", "Airport", "Time"]
merge_left_origin = ["Year", "Month", "DayofMonth", "Origin", "DepHour"]
merge_left_dest = ["Year", "Month", "DayofMonth", "Dest", "ArrHour"]

merge_drop = ["Date_Origin", "Airport_Origin", "Time_Origin", "Date_Dest", "Airport_Dest", "Time_Dest"]


In [ ]:
# To Extract hour from given time
def hour(x):
    if len(str(x)) > 2:
        return int(str(x)[-3::-1][::-1]+"00")
    return 100

# To convert given JSON to DF
def jsonToDF(weather, month, year, airport):
    temp = []
    for date in weather:
        for hour in date["hourly"]:
            row = [weather.index(date)+1, month, year, airport]
            for key in weatherKeys:
                row.append(hour[key])
            temp.append(row)
    return pd.DataFrame(temp, columns=weatherCols)

## 3. Reading Flight Data 


In [ ]:
# Reading Flight Data (2 min)
flightDF = dd.read_csv(os.path.join(basePath, 'flight-data', '*', '*', '*.csv'), usecols=flightCols)
flightDF = flightDF.compute()

# Retaining only required airports
flightDF = flightDF[flightDF["Origin"].isin(airports)].dropna()
flightDF = flightDF[flightDF["Dest"].isin(airports)]

# Setting times to int
flightDF = flightDF.astype({'ArrTime': np.int64, 'DepTime': np.int64})

# Resetting indices
flightDF = flightDF.reset_index(drop=True)

# Calculating Time as hour alone
flightDF["DepHour"] = pd.Series([hour(x) for x in flightDF["DepTime"]])
flightDF["ArrHour"] = pd.Series([hour(x) for x in flightDF["ArrTime"]])


# Saving FlightDF
flightDF.to_csv(os.path.join(basePath, "flight-data", "FlightData.csv"), index=False) 

flightDF.shape

(1851436, 19)

In [ ]:
flightDF = pd.read_csv(os.path.join(basePath, "flight-data", "FlightData.csv"))
flightDF.head()

,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,DepHour,ArrHour
0,2016,1,1,1,2016-01-01,14747,SEA,12478,JFK,745,741,0.0,0.0,1602,1610,8.0,0.0,700,1600
1,2016,1,1,2,2016-01-02,14747,SEA,12478,JFK,745,737,0.0,0.0,1602,1613,11.0,0.0,700,1600
2,2016,1,1,3,2016-01-03,14747,SEA,12478,JFK,745,743,0.0,0.0,1602,1547,0.0,0.0,700,1500
3,2016,1,1,4,2016-01-04,14747,SEA,12478,JFK,745,737,0.0,0.0,1602,1551,0.0,0.0,700,1500
4,2016,1,1,5,2016-01-05,14747,SEA,12478,JFK,710,708,0.0,0.0,1527,1524,0.0,0.0,700,1500


## 4. Reading Weather Data


In [ ]:
# Reading Weather Data (5 min)
weatherList = []
for airport in airports:
    for year in years: 
        for month in months:
            # Path to each JSON fi
            path = os.path.join(basePath, "weather", airport+"/"+str(year)+"-"+str(month)+".json")
            with open(path) as json_file:
                weather = json.load(json_file)["data"]["weather"]
            # Append each DF into weatherList 
            weatherList.append(delayed(jsonToDF)(weather, month, year, airport).compute())

# Merging DFs into weatherDF
weatherDF = pd.concat(weatherList, ignore_index=True)
weatherDF["Time"] = [int(x) + 100 for x in weatherDF["Time"]]
weatherDF.to_csv(os.path.join(basePath, "weather", "WeatherData.csv"), index=False) # Saving data into weatherData.csv
weatherDF.shape

(263160, 17)

In [ ]:
weatherDF = pd.read_csv(os.path.join(basePath, "weather", "WeatherData.csv"))
weatherDF.head()

,Date,Month,Year,Airport,WindSpeedKmph,WindDirDegree,WeatherCode,precipMM,Visibility,Pressure,Cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,Humidity,Time
0,1,1,2016,ATL,11,318,176,0.1,10,1023,100,47,17,49,46,91,100
1,1,1,2016,ATL,13,317,176,0.0,10,1023,100,44,22,46,42,92,200
2,1,1,2016,ATL,14,315,122,0.0,10,1023,100,41,26,43,38,92,300
3,1,1,2016,ATL,16,314,122,0.0,10,1023,100,38,30,40,33,93,400
4,1,1,2016,ATL,17,314,122,0.0,10,1023,100,38,30,40,33,93,500


## 5. Merging Flight and Weather Data

In [ ]:
# Adding flight origin weather details
data = flightDF.merge(weatherDF, left_on=merge_left_origin, right_on=merge_right)

# Adding flight destination weather details
data = data.merge(weatherDF, left_on=merge_left_dest, right_on=merge_right, suffixes=("_Origin", "_Dest"))

# Dropping redundant columns
data = data.drop(merge_drop, axis=1)

# Saving into Data.csv
data.to_csv(os.path.join(basePath, "dataset.csv"), index=False)
data.shape

(1851436, 43)

In [ ]:
data = pd.read_csv(os.path.join(basePath, "dataset.csv"))
data.head()

,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,DepHour,ArrHour,WindSpeedKmph_Origin,WindDirDegree_Origin,WeatherCode_Origin,precipMM_Origin,Visibility_Origin,Pressure_Origin,Cloudcover_Origin,DewPointF_Origin,WindGustKmph_Origin,tempF_Origin,WindChillF_Origin,Humidity_Origin,WindSpeedKmph_Dest,WindDirDegree_Dest,WeatherCode_Dest,precipMM_Dest,Visibility_Dest,Pressure_Dest,Cloudcover_Dest,DewPointF_Dest,WindGustKmph_Dest,tempF_Dest,WindChillF_Dest,Humidity_Dest
0,2016,1,1,1,2016-01-01,14747,SEA,12478,JFK,745,741,0.0,0.0,1602,1610,8.0,0.0,700,1600,7,106,113,0.0,10,1030,0,24,8,34,29,67,21,254,113,0.0,10,1016,0,33,24,42,33,74
1,2016,1,1,1,2016-01-01,13204,MCO,12478,JFK,1330,1408,38.0,1.0,1603,1628,25.0,1.0,1400,1600,8,227,122,0.1,8,1019,79,72,10,79,78,83,21,254,113,0.0,10,1016,0,33,24,42,33,74
2,2016,1,1,1,2016-01-01,13204,MCO,12478,JFK,1445,1445,0.0,0.0,1720,1659,0.0,0.0,1400,1600,8,227,122,0.1,8,1019,79,72,10,79,78,83,21,254,113,0.0,10,1016,0,33,24,42,33,74
3,2016,1,1,1,2016-01-01,12892,LAX,12478,JFK,855,853,0.0,0.0,1712,1650,0.0,0.0,800,1600,12,29,113,0.0,10,1020,1,30,15,50,47,47,21,254,113,0.0,10,1016,0,33,24,42,33,74
4,2016,1,1,1,2016-01-01,12892,LAX,12478,JFK,840,842,2.0,0.0,1708,1626,0.0,0.0,800,1600,12,29,113,0.0,10,1020,1,30,15,50,47,47,21,254,113,0.0,10,1016,0,33,24,42,33,74
